In [1]:
import sys
sys.path.append('..')
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sympy import simplify_logic

from deep_logic.utils.base import validate_network
from deep_logic.utils.relunn import get_reduced_model, prune_features
from deep_logic import fol
import deep_logic as dl

torch.manual_seed(0)
np.random.seed(0)

In [2]:
x = pd.read_csv('dsprites_c_train.csv', index_col=0)
y = pd.read_csv('dsprites_y_train.csv', index_col=0)

In [3]:
base_concepts = ['color', 'shape', 'scale', 'rotation', 'x_pos', 'y_pos']
base_concepts

['color', 'shape', 'scale', 'rotation', 'x_pos', 'y_pos']

In [4]:
colors = ['white']
shapes = ['square', 'ellipse', 'heart']
scale = ['very small', 'small', 's-medium', 'b-medium', 'big', 'very big']
rotation = ['0°', '5°', '10°', '15°', '20°', '25°', '30°', '35°']
x_pos = ['x0', 'x2', 'x4', 'x6', 'x8', 'x10', 'x12', 'x14', 'x16', 'x18', 'x20', 'x22', 'x24', 'x26', 'x28', 'x30']
y_pos = ['y0', 'y2', 'y4', 'y6', 'y8', 'y10', 'y12', 'y14', 'y16', 'y18', 'y20', 'y22', 'y24', 'y26', 'y28', 'y30']
concepts = colors + shapes + scale + rotation + x_pos + y_pos

In [5]:
x_train = torch.tensor(x.values, dtype=torch.float)
y_train = torch.zeros((y.shape[0], 3), dtype=torch.long)
y_train[:, 0] = torch.tensor(y.values[:, 0] + y.values[:, 8])# + y.values[:, 12] + y.values[:, 14])
#y_train[:, 1] = torch.tensor(y.values[:, 9])
y_train[:, 1] = 1-(y_train[:, 0])# + y_train[:, 1])
y_train = torch.tensor(y.values[:, :3])
y_train = torch.argmax(y_train, dim=1)
x_test = x_train
n_classes = len(torch.unique(y_train))
print(x_train.shape)
print(y_train.shape)
print(n_classes)
x

torch.Size([5530, 50])
torch.Size([5530])
3


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5525,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5526,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5527,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5528,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
y_train = torch.zeros((y.shape[0], 3), dtype=torch.float)
y_train[:, 0] = torch.tensor(y.values[:, 0], dtype=torch.float)
y_train[:, 1] = torch.tensor(y.values[:, 7], dtype=torch.float)
y_train[:, 2] = 1-(y_train[:, 0] + y_train[:, 1])
x_test = x_train
#n_classes = len(torch.unique(y_train))
n_classes = 3
print(n_classes)
y_train

3


tensor([[1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        ...,
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]])

In [7]:
y_train.sum(dim=0)

tensor([ 321.,  302., 4907.])

In [8]:
torch.manual_seed(0)
np.random.seed(0)

layers = [
    torch.nn.Linear(x_train.size(1), 20 * n_classes),
    torch.nn.LeakyReLU(),
    dl.nn.XLinear(20, 10, n_classes),
    torch.nn.LeakyReLU(),
    dl.nn.XLinear(10, 5, n_classes),
    torch.nn.LeakyReLU(),
    dl.nn.XLinear(5, 1, n_classes),
    torch.nn.Softmax(),
]
model = torch.nn.Sequential(*layers)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_form = torch.nn.BCELoss()
model.train()
need_pruning = True
for epoch in range(2000):
    # forward pass
    optimizer.zero_grad()
    y_pred = model(x_train)
    # Compute Loss
    loss = loss_form(y_pred, y_train)

    for module in model.children():
        if isinstance(module, torch.nn.Linear):
            loss += 0.001 * torch.norm(module.weight, 1)
            break

    # backward pass
    loss.backward()
    optimizer.step()
    
    if epoch > 1000 and need_pruning:
        prune_features(model, n_classes)
        #need_pruning = False

    # compute accuracy
    if epoch % 500 == 0:
        y_pred_d = torch.argmax(y_pred, dim=1)
        y_train_d = torch.argmax(y_train, dim=1)
        accuracy = y_pred_d.eq(y_train_d).sum().item() / y_train.size(0)
        print(f'Epoch {epoch}: train accuracy: {accuracy:.4f}')

Epoch 0: train accuracy: 0.0580


/home/pietro/anaconda3/envs/dev/lib/python3.7/site-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 500: train accuracy: 1.0000
Epoch 1000: train accuracy: 1.0000
Epoch 1500: train accuracy: 1.0000


# Local explanations

In [9]:
np.set_printoptions(precision=2, suppress=True)
outputs = []
for i, (xin, yin) in enumerate(zip(x_train, y_train)):
    model_reduced = get_reduced_model(model, xin)
    for module in model_reduced.children():
        if isinstance(module, torch.nn.Linear):
            wa = module.weight.detach().numpy()
            break
    output = model_reduced(xin)
    
    pred_class = torch.argmax(output)
    true_class = torch.argmax(y_train[i])

    # generate local explanation only if the prediction is correct
    if pred_class.eq(true_class) and true_class < 2:
        local_explanation = fol.relunn.explain_local(model, x_train, y_train, xin, concepts)
        print(f'Input {(i+1)}')
        print(f'\tx={xin.detach().numpy()}')
        print(f'\ty={y_train[i].detach().numpy()}')
        print(f'\ty={output.detach().numpy()}')
        #print(f'\tw={wa}')
        print(f'\tExplanation: {local_explanation}')
        print()
    outputs.append(output)
    if i > 30:
        break

Input 1
	x=[1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0.]
	y=[1. 0. 0.]
	y=[0.45 0.   0.  ]
	Explanation: square & ~ellipse & very small & ~small

Input 7
	x=[1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
	y=[1. 0. 0.]
	y=[0.45 0.   0.  ]
	Explanation: square & ~ellipse & very small & ~small

Input 14
	x=[1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1.]
	y=[1. 0. 0.]
	y=[0.45 0.   0.  ]
	Explanation: square & ~ellipse & very small & ~small

Input 22
	x=[1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1.]
	y=[1. 0. 0.]
	y=[0.45 0.   0.  ]
	Explanation: square & ~ellipse & ver

# Combine local explanations

In [10]:
y_train_d = torch.argmax(y_train, dim=1)
global_explanation, predictions, counter = fol.combine_local_explanations(model, x_train, y_train,
                                                                          topk_explanations=10, target_class=0,
                                                                          concept_names=concepts)


y2 = torch.argmax(y_train, dim=1) == 0
accuracy = sum(predictions == y2.detach().numpy().squeeze()) / len(predictions)
print(f'Accuracy using the formula "{global_explanation}": {accuracy:.4f}')

Accuracy using the formula "square & very small & ~ellipse & ~small": 1.0000


In [11]:
counter.most_common(4)

[('square & ~ellipse & very small & ~small', 321)]

In [12]:
y_train_d = torch.argmax(y_train, dim=1)
global_explanation, predictions, counter = fol.combine_local_explanations(model, x_train, y_train,
                                                                          topk_explanations=10, target_class=1,
                                                                          concept_names=concepts)


y2 = torch.argmax(y_train, dim=1) == 1
accuracy = sum(predictions == y2.detach().numpy().squeeze()) / len(predictions)
print(f'Accuracy using the formula "{global_explanation}": {accuracy:.4f}')

Accuracy using the formula "ellipse & small & ~square & ~very small": 1.0000


In [13]:
counter.most_common(4)

[('~square & ellipse & ~very small & small', 302)]